[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/instructlab/examples/blob/ux-notebook-reviewer/ux-notebook-reviewer/ux_notebook_reviewer.ipynb)

# UX Notebook Reviewer

## Goal & Objective
This notebook allows you to upload and evaluate other Jupyter Notebooks based on UX best practices. It checks for clarity, structure, readability, and completeness — helping notebook authors improve user experience.
The goal is to automate the review process using both structured logic and, eventually, an AI model.

### Usage Instructions:

0. **Install Required Packages:** What you will need. 
1. **Upload your Notebook:** Use the file uploader to select the `.ipynb` file you wish to review.
2. **Automatted UX Review:** Execute the code cells in sequence. The notebook will parse your uploaded file and apply the UX checklist.
3. **Review the Report:** A markdown table will be generated, detailing the status of each guideline and providing suggestions where applicable.

---

## 0. Install Required Pacakges
Before running any of the automation or parsing cells, make sure you have these core dependencies:

- **ipywidgets** (≥7.6.0): for interactive loading indicators  
- **nbformat**   (≥5.1.3): to parse `.ipynb` files into Python objects  
- **openai**     (≥0.27.0): your Ollama-shim client library  

In [ ]:
# Install required packages (run once)
!pip install --upgrade ipywidgets nbformat openai pandas matplotlib

## 1. Upload Notebook File
Creates an upload button where you can select a .ipynb file. Once uploaded, the file will be held in memory and ready for review.

---

In [ ]:
from ipywidgets import FileUpload
from IPython.display import display

upload = FileUpload(accept='.ipynb', multiple=False)
display(upload)

---

## 2. Automated UX Review

Maintaining consistent usability across Jupyter notebooks can be time-consuming and subjective. This automated evaluation:

1. Loads and validates your uploaded notebook  
2. Runs each guideline from our UX template against the notebook  
3. Produces an objective Markdown table of results (✅/🔹/❌/–)  
4. Reports the total elapsed time for transparency  

Run this cell to instantly uncover UX strengths and weaknesses and ensure every notebook meets our quality standards without manual effort.  

In [ ]:
# ─── Cell: Automated UX Review w/ Live Timer & Elapsed Header ───
import time, threading
import nbformat
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# ——— Constants ——————————————————————————————————————————————
LEGEND_MD = """**Legend: **  
✅ 90–100% compliance (excellent)  
🔹 60–89% compliance (good but needs a little work)  
❌ 0–59% compliance (major issue)  
– not applicable  
"""

TABLE_TEMPLATE = """| Guideline      | Status | Suggestion                               |
|:-----------------------|:----------|:--------------------------------------------------|
| Clear Header           |           |                                                   |
| Goal/Objective Present |           |                                                   |
| Setup & Pre-Requisites |           |                                                   |
| Markdown Usage         |           |                                                   |
| Avoid Hardcoding       |           |                                                   |
| Inline Code Comments   |           |                                                   |
| Next Steps & Conclusion|           |                                                   |
"""

# ——— 1) Validate upload and load notebook —————————————————————
notebook_data, notebook_name = load_notebook(upload)
if notebook_data is None:
    clear_output(wait=True)
    display(Markdown(f"❌ {notebook_name}"))
    raise RuntimeError("Please upload a valid notebook above.")
display(Markdown(f"✅ Loaded **{notebook_name}** with **{len(notebook_data.cells)}** cells."))

# ——— 2) Prepare prompt —————————————————————————————————————
notebook_text = extract_notebook_text(notebook_data)
review_prompt = f"""
Here is a Jupyter notebook for review:

{notebook_text}

{LEGEND_MD}

Return *only* this Markdown table, filled in:

{TABLE_TEMPLATE}
"""

# ——— 3) Start live timer + spinner ————————————————————————
start_time  = time.time()
timer_label = widgets.Label("Elapsed: 0.0s")
spinner     = widgets.HTML("<b>🧠 Evaluating the notebook…</b>")
display(widgets.HBox([spinner, timer_label]))

stop_flag = threading.Event()
def _tick():
    while not stop_flag.is_set():
        elapsed = time.time() - start_time
        timer_label.value = f"Elapsed: {elapsed:.1f}s"
        time.sleep(0.1)

thread = threading.Thread(target=_tick, daemon=True)
thread.start()

# ——— 4) Invoke the LLM ———————————————————————————————————
client   = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
response = client.chat.completions.create(
    model="llama3.1:8b-instruct-fp16",
    messages=[
        {"role": "system", "content": "You are a helpful UX reviewer of Jupyter Notebooks."},
        {"role": "user",   "content": review_prompt},
    ],
    temperature=0.3,
)

# ——— 5) Stop timer, join, clear output ——————————————————————
stop_flag.set()
thread.join(timeout=0.5)
elapsed = time.time() - start_time
clear_output(wait=True)

# ─── 6) Render header + table ─────────────────────────────
display(Markdown(f"## UX Review of **{notebook_name}** (took {elapsed:.1f}s)\n\n{LEGEND_MD}"))

table_lines = [
    ln for ln in response.choices[0].message.content.splitlines()
    if ln.lstrip().startswith("|")
]
display(Markdown("\n".join(table_lines)))

# ─── 7) Bullet‐list of suggestions ────────────────────────
suggestions = []
# skip header row (0) and separator row (1)
for row in table_lines[2:]:
    cols = [c.strip() for c in row.strip("|").split("|")]
    name, icon, sugg = cols[0], cols[1], cols[2]
    if sugg and sugg != "-":
        suggestions.append(f"- **{name}** ({icon}): {sugg}")

if suggestions:
    display(Markdown("### Suggestions\n" + "\n".join(suggestions)))

# Next Steps

- Review and correct any 🔹/❌ items above.  
- Commit this notebook and rerun the review to confirm compliance.  
- Integrate into your CI pipeline for automated checks on every PR.  
